Data Collection Managment

KAYAK: Plan your trip

Goals
- Scrape data from destinations
- Get weather data from each destination
- Get hotel's info about each destination
- Store all the information above in a data lake
- Extract , transform and load cleaned data from your datala to a data warehouse

1. Project Set-up

In [109]:
#Import modules 
import pandas as pd
import os
from dotenv import load_dotenv

In [ ]:
pd.set_option('display.max_columns',None)

In [5]:
#List of top cities
top_cities = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
    "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", 
    "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", 
    "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", 
    "Montauban", "Biarritz", "Bayonne", "La Rochelle"
]

2. Get Weather Data (API)

In [ ]:
#Module for API requests
import requests

2.1 Test API - GPS coodinates

In [8]:
#API request TEST 
response = requests.get("https://nominatim.openstreetmap.org")
print(response)

<Response [200]>


2.2 Test API OpenWeather

In [7]:
#Open weather api key
API_KEY = os.environ.get('OPENWEATHER_API_KEY')

In [47]:
# Test API openweather 
res = requests.get(f'https://api.openweathermap.org/data/3.0/onecall?lat=33.44&lon=-94.04&appid={API_KEY}&units=metric')
data_weather = res.json()['daily']
data_weather

[{'dt': 1738692000,
  'sunrise': 1738674589,
  'sunset': 1738713024,
  'moonrise': 1738687560,
  'moonset': 0,
  'moon_phase': 0.23,
  'summary': 'The day will start with clear sky through the late morning hours, transitioning to partly cloudy',
  'temp': {'day': 21.03,
   'min': 16.08,
   'max': 21.74,
   'night': 16.67,
   'eve': 17.22,
   'morn': 16.42},
  'feels_like': {'day': 21.46, 'night': 16.9, 'eve': 17.48, 'morn': 16.68},
  'pressure': 1021,
  'humidity': 87,
  'dew_point': 18.75,
  'wind_speed': 3.19,
  'wind_deg': 97,
  'wind_gust': 6.74,
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04d'}],
  'clouds': 71,
  'pop': 0,
  'uvi': 2.12},
 {'dt': 1738778400,
  'sunrise': 1738760942,
  'sunset': 1738799482,
  'moonrise': 1738776240,
  'moonset': 1738738800,
  'moon_phase': 0.25,
  'summary': 'Expect a day of partly cloudy with rain',
  'temp': {'day': 19.84,
   'min': 16.14,
   'max': 22.43,
   'night': 19.26,
   'eve': 20.7,
  

2.3 Set-up API calls in order to extract gps coordinates for each cities and the weather for the next 7 days

In [9]:
weather_data = []
base_url = 'https://nominatim.openstreetmap.org/search'
headers = {"User-Agent": "MyApp/1.0"}
api_count =0
for city in top_cities:
    params = {"q": city, "format": "json"}
    res = requests.get(base_url, params=params, headers=headers)
    lat = res.json()[0]["lat"]
    lon = res.json()[0]["lon"]
    
    #data_weather 
    response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={API_KEY}&units=metric")
    data_weather=response.json()['daily']
    for i, day in enumerate(data_weather[1:8]):

        timestamp = day['dt']
        temp_day = day['temp']['day']
        summary = day['summary']
        num_day = i + 1
        humidity =day ['humidity']
        dew_point =day ['dew_point']
        weather_main = day['weather'][0]['main']
        weather_descrip = day['weather'][0]['description']
        clouds =day['clouds']
        wind_speed = day['wind_speed']
        pop = day['pop']
        
        print(f'Fecth success for {city}. Status code:{response}')
        api_count += 1
        weather_data.append([
            num_day,timestamp,city,lat,lon,temp_day,humidity,dew_point,
            weather_main,weather_descrip,clouds,wind_speed,pop])
        
        
print(f'\n Total API calls:{api_count}')   

Fecth success for Mont Saint Michel. Status code:<Response [200]>
Fecth success for Mont Saint Michel. Status code:<Response [200]>
Fecth success for Mont Saint Michel. Status code:<Response [200]>
Fecth success for Mont Saint Michel. Status code:<Response [200]>
Fecth success for Mont Saint Michel. Status code:<Response [200]>
Fecth success for Mont Saint Michel. Status code:<Response [200]>
Fecth success for Mont Saint Michel. Status code:<Response [200]>
Fecth success for St Malo. Status code:<Response [200]>
Fecth success for St Malo. Status code:<Response [200]>
Fecth success for St Malo. Status code:<Response [200]>
Fecth success for St Malo. Status code:<Response [200]>
Fecth success for St Malo. Status code:<Response [200]>
Fecth success for St Malo. Status code:<Response [200]>
Fecth success for St Malo. Status code:<Response [200]>
Fecth success for Bayeux. Status code:<Response [200]>
Fecth success for Bayeux. Status code:<Response [200]>
Fecth success for Bayeux. Status cod

2.4 Save weather data to acsv file

In [10]:
df_weather = pd.DataFrame(weather_data)

#add column headers to the Dataframe
df_weather =df_weather.rename(columns={
    0:'num_day',1:'date_time',2:'city',3:'lat',4:'lon',5:'temp_celsius',
    6:'humidity',7:'dew_point',8:'weather_main',9:'weather_descrip',10:'clouds',11:'wind_speed',12:'pop'})

In [11]:
df_weather.head()

,num_day,date_time,city,lat,lon,temp_celsius,humidity,dew_point,weather_main,weather_descrip,clouds,wind_speed,pop
0,1,1738756800,Mont Saint Michel,48.6359541,-1.511459954959514,9.64,73,4.78,Clouds,scattered clouds,31,3.55,0.86
1,2,1738843200,Mont Saint Michel,48.6359541,-1.511459954959514,8.07,67,2.11,Clouds,broken clouds,66,7.11,0.00
2,3,1738929600,Mont Saint Michel,48.6359541,-1.511459954959514,4.47,69,-0.78,Clouds,overcast clouds,99,7.12,0.00
3,4,1739016000,Mont Saint Michel,48.6359541,-1.511459954959514,6.29,63,-0.43,Rain,light rain,71,4.49,0.46
4,5,1739102400,Mont Saint Michel,48.6359541,-1.511459954959514,6.21,59,-1.30,Clouds,broken clouds,75,3.68,0.00


In [29]:
#Save weather data to csv file
df_weather.to_csv('src/weather_data.csv')

3. Web Scraping

Scrape Booking.com website to get the following information of each cities in the list:

- Hotel name
- Url to its booking.com page
- Its coordinates: latitude and longitude
- Score given by the website users
- Text description of the hotel

3.1 Run booking_spider Crawler to extract hotel information

In [12]:
! python booking_spider.py

2025-02-04 20:00:26 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-02-04 20:00:26 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 25.0.0 (OpenSSL 3.4.0 22 Oct 2024), cryptography 44.0.0, Platform Windows-11-10.0.26100-SP0
2025-02-04 20:00:26 [scrapy.addons] INFO: Enabled addons:
[]
2025-02-04 20:00:26 [scrapy.extensions.telnet] INFO: Telnet Password: accae00478e67d76
2025-02-04 20:00:26 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2025-02-04 20:00:26 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 'INFO',
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/91.0.4472.124 Sa

3.2 Save hotel information to csv

In [69]:
# Import modules to load json file and convert it to csv file
import json
import csv

In [71]:
# File paths
json_file, csv_file = "src/hotels.json", "src/hotels.csv"

# Read JSON file and convert to list
with open(json_file, "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file]

# Write to CSV
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"CSV file saved as {csv_file}")

CSV file saved as src/hotels.csv


4. Data Storage using Boto3

In [72]:
# Boto3 installation to acces datalake
! pip install boto3


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


4.1 Set up access to AWS DSK

In [131]:
#Assigned secret key and access id
load_dotenv()
AWS_ACCESS_SECRET_KEY = os.environ.get('AWS_ACCESS_SECRET_KEY')
AWS_ACCESS = os.environ.get('AWS_ACCESS')

In [133]:
# Create an instance of boto3.Session to connect to AWS with access id and secret key
import boto3
session = boto3.Session(aws_access_key_id= AWS_ACCESS, 
                        aws_secret_access_key= AWS_ACCESS_SECRET_KEY)

In [134]:
# Create a variable called 's3' to connect to a specific AWS sercice which is 's3'
s3 = session.resource('s3')

4.2 Create bucket in S3 Datalake and store raw data files

In [ ]:
# Create a variable called 'bucket' and create a new S3 bucket 
bucket = s3.create_bucket(Bucket ="jedha-8568-kayak-8557-project-2025-98746")

In [139]:
# Upload the raw data files using put_objet() function in the bucket
put_object1 = bucket.put_object(Key='hotels.csv', Body='src/hotels.csv')
put_object2 = bucket.put_object(Key='weather_data.csv', Body='src/weather_data.csv')